# 0. Imports

In [2]:
# Basic imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

import math

import datasus_dbc # Used to decompress the DBC file from DATASUS
import dbfread # Used to read the decompressed DBF

from IPython.core.display import HTML
from IPython.display import Image

In [3]:
# Set the option to display all columns in the DataFrame
pd.set_option('display.max_columns', None)

In [4]:
def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

jupyter_settings()

# 1. Class RT_APAC

In [5]:
class RT_APAC:
      
    def concat_all_data(path_files, state): # Final dictionary to hold the results  

        # Create a full data DataFrame to hold all data
        full_data = pd.DataFrame()            
        
        # Path to the directory containing the DBC files
        path_files = os.path.join(path_files, state)  # Adjust the path as needed

        # Check all paths in the directory
        for path in os.listdir(path_files):

            # Create an empty DataFrame to hold all data
            all_data = pd.DataFrame()

            # Print the path being processed
            print("Processing path:", path, end=' from -> ')            

            # Construct the full path to the directory            
            path_destiny = os.path.join(path_files, path)
            print(path_destiny)            
        
            # Check all files in the path
            try:
                for file in os.listdir(path_destiny):                
                
                    # Construct the full path to the file
                    file_to_read = os.path.join(path_destiny, file)   
                
                    # Decompress the DBC file to a temporary DBF file
                    datasus_dbc.decompress(file_to_read, "output.dbf")

                    # Read the decompressed DBF file using dbfread
                    dbf = dbfread.DBF('output.dbf', encoding='latin1')
                    #df = pd.DataFrame(iter(dbf))

                    # Now you can work with the DataFrame
                    df = pd.DataFrame(iter(dbf))  # Convert to DataFrame

                    for col in df.columns:
                        # Convert columns to string type
                        df[col] = df[col].astype(str)                

                    # Concatenate the DataFrame to the main DataFrame                
                    all_data = pd.concat([all_data, df], ignore_index=True)            
                
                    # Construct the full path to save the CSV file
                    csv_destiny = os.path.join(path_files, f"{path}.csv") 
            except NotADirectoryError:
                print(f"Error: {path_destiny} is not a directory or does not exist.")
                pass               
        
            # Concatenate the all_data DataFrame to the full_data DataFrame
            full_data = pd.concat([full_data, all_data], ignore_index=True)
        

        # Save the full_data DataFrame to a CSV file
        csv_destiny = os.path.join(path_files, f"{state}.csv")
        full_data.to_csv(csv_destiny, index=False)

In [7]:
data = RT_APAC.concat_all_data("C:\\Users\\175 MX\\Documents\\Gustavo\\datasus\\files\\data_rt_states\\states\\", "sp")

Processing path: 08 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\08
Processing path: 09 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\09
Processing path: 10 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\10
Processing path: 11 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\11
Processing path: 12 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\12
Processing path: 13 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\13
Processing path: 14 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\14
Processing path: 15 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\15
Processing path: 16 from -> C:\Users\175 MX\Documents\Gustavo\datasus\files\data_rt_states\states\sp\16
Processing path: 17 from -> C:\Users\175 MX\Documents\Gustavo\da